# 使用手寫辨識資料集, 觀察 PCA 算法

# [作業目標]
- 試著模仿範例寫法, 以變更參數後的 PCA + 邏輯斯迴歸判斷手寫辨識資料集, 觀察不同 component 下正確率的變化

# [作業重點]
- 以手寫辨識資料集, 觀察變更參數後的 PCA 算法, 在取不同 component 時, PCA 解釋度與分類正確率如何變化 (In[5], Out[5])

In [36]:
# 載入套件
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings("ignore")

In [37]:
# 定義 PCA 與隨後的羅吉斯迴歸函數
# penalty 改為 L1, max_iter 改為 100
logistic = SGDClassifier(loss='log', penalty='l1', max_iter=100, tol=1e-5, random_state=0)
pca = PCA()
pipe = Pipeline(steps=[('pca', pca), ('logistic', logistic)])

# 載入手寫數字辨識集
digits = datasets.load_digits()
X_digits = digits.data
y_digits = digits.target

# 作業
* 將參數 penalty 改為 L1, max_iter 改為 100, 觀察 PCA 與羅吉斯迴歸做手寫辨識的效果

In [ ]:
# 先執行 GridSearchCV 跑出最佳參數
params = {'pca__n_components':[4,9,16,25,36,49,64],
    'logistic__alpha': np.logspace(-4, 4, 5)}
result = GridSearchCV(pipe,params,n_jobs = -1,cv = 5)
result.fit(X_digits,y_digits)
print(f"best score :{result.best_score_}")
print(result.best_params_)

In [ ]:
# 繪製不同 components 的 PCA 解釋度
pca.fit(X_digits,y_digits)
fig,(ax1,ax2) = plt.subplots(nrows = 2,sharex = True,figsize = (6,6))
ax1.plot(pca.explained_variance_ratio_)
df = pd.DataFrame(result.cv_results_)
grouped = df.groupby('param_pca__n_components').apply(lambda g: g.nlargest(1, 'mean_test_score'))

grouped.plot(x ='param_pca__n_components', y = 'mean_test_score', yerr='std_test_score', legend=False, ax=ax2)
plt.show()

In [ ]:
# 繪製不同採樣點的分類正確率
df = pd.DataFrame(result.cv_results_)
grouped = df.groupby('param_pca__n_components').apply(lambda g: g.nlargest(1, 'mean_test_score'))

grouped.plot(x ='param_pca__n_components', y = 'mean_test_score', yerr='std_test_score', legend=False, ax=ax1)
plt.show()